# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298445176911                   -0.85    5.4
  2   -8.300206526860       -2.75       -1.25    1.0
  3   -8.300444368797       -3.62       -1.89    3.5
  4   -8.300462620410       -4.74       -2.76    1.4
  5   -8.300464318847       -5.77       -3.04    7.9
  6   -8.300464463093       -6.84       -3.20    1.1
  7   -8.300464551793       -7.05       -3.34    2.4
  8   -8.300464598829       -7.33       -3.48    1.0
  9   -8.300464632538       -7.47       -3.68    1.0
 10   -8.300464635476       -8.53       -3.74    1.4
 11   -8.300464641711       -8.21       -3.95    3.8
 12   -8.300464643955       -8.65       -4.50    2.2


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67498285396                   -0.70    7.6
  2   -16.67868351677       -2.43       -1.14    1.5
  3   -16.67924473372       -3.25       -1.86    2.9
  4   -16.67928145261       -4.44       -2.76    2.6
  5   -16.67928605647       -5.34       -3.23    5.0
  6   -16.67928621058       -6.81       -3.49    4.6
  7   -16.67928622104       -7.98       -3.94    1.0
  8   -16.67928622398       -8.53       -4.63    1.1


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32685087673                   -0.56    7.4
  2   -33.33436229445       -2.12       -1.00    1.1
  3   -33.33601471967       -2.78       -1.72    6.2
  4   -33.33689305283       -3.06       -2.53    6.6
  5   -33.33694309992       -4.30       -3.17    5.6
  6   -33.33694388464       -6.11       -3.95    5.1
  7   -33.33694392617       -7.38       -4.49    5.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298474634625                   -0.85    5.2
  2   -8.300275139982       -2.74       -1.59    1.0
  3   -8.300460688557       -3.73       -2.68    5.1
  4   -8.300440709491   +   -4.70       -2.57   10.5
  5   -8.300464510838       -4.62       -3.72    2.0
  6   -8.300464599824       -7.05       -3.98    4.2
  7   -8.300464643268       -7.36       -4.79    3.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32394328418                   -0.56    7.1
  2   -33.30479709371   +   -1.72       -1.26    1.0
  3   -6.180177772491   +    1.43       -0.35    8.9
  4   -33.32064275453        1.43       -1.63    7.0
  5   -33.16385077063   +   -0.80       -1.27    4.6
  6   -29.86026900299   +    0.52       -0.80    4.9
  7   -33.32811523322        0.54       -2.03    4.8
  8   -33.33350669127       -2.27       -2.04    2.8
  9   -33.33507935766       -2.80       -2.14    2.2
 10   -33.33630585948       -2.91       -2.61    2.0
 11   -33.33672495081       -3.38       -2.83    2.0
 12   -33.33681991953       -4.02       -2.98    2.2
 13   -33.33693433381       -3.94       -3.45    3.0
 14   -33.33694350266       -5.04       -4.02    3.5


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.